Apply scVelo to simulated data
---------------


In this file we want to apply scVelo to our simulated data. This code follows the struture of the notebook of scVelo **Dynamical Modeling** (https://github.com/theislab/scvelo_notebooks/blob/master/DynamicalModeling.ipynb).

First of all, the input data for scVelo are two count matrices of pre-mature (unspliced) and mature (spliced) abundances,
which can be obtained from standard sequencing protocols.


**Import packages**


In [ ]:
# update to the latest version, if not done yet.
!pip install scvelo --upgrade --quiet

In [ ]:
import scanpy
import scvelo as scv
import numpy as np
import scipy.sparse
import pandas as pd
import anndata 

In [ ]:
import os
import warnings

import numpy as np
import pandas as pd
from scipy.optimize import minimize

import matplotlib.pyplot as pl
from matplotlib import rcParams
from scipy.sparse import issparse


from scvelo import logging as logg
from scvelo import settings
from scvelo.core import get_n_jobs, parallelize
from scvelo.preprocessing.moments import get_connectivities
from scvelo.tools.utils import make_unique_list, test_bimodality
from scvelo import logging as logg
from scvelo.core import clipped_log, invert, SplicingDynamics
from scvelo.preprocessing.moments import get_connectivities
from scvelo.tools.utils import make_dense, round


In [ ]:
scv.settings.verbosity = 3  # show errors(0), warnings(1), info(2), hints(3)
scv.settings.presenter_view = True  # set max width size for presenter view
scv.settings.set_figure_params('scvelo')  # for beautified visualization



**Read your data**



Read your data file (loom, h5ad, csv, ...) and store the data matrix (``adata.X``), annotation of cells / observations (``adata.obs``) and genes / variables (``adata.var``), unstructured annotation such as graphs (``adata.uns``) and additional data layers where spliced and unspliced counts are stored (``adata.layers``) .


In our case, we are going to load the data we have already simulated using file *dataSimulationScVelo.r*.
We can decide here which type of simulation we want to load.
We can set 
- **model** (*NormInd*, for Normal Independent model, or *Demings*, for Deming Residuals model)
- **typeSW** (*SW1* for common switching time, or *SW2* for cluster-specific switching points)
- **typeT** (**T1** vs **T2** vs **T3**, determining the number of subgroups)
- **typeD** (**D1**, for Poisson distributed, and **D4**: Negative Binomial distributed bayVel data)

In [ ]:
# --- input parameters
pathToYourDirectory = "pathToYourDirectory"
model = "Demings"
typeSW = "SW1"
typeT = "T1"
typeD = "D1"


# --- Set the name of the simulation
n_genes = "2000"
nameSIM =  str(typeSW) + "-" + str(typeT) + "-" +str(typeD) 
path = pathToYourDirectory + "/simulations/" + nameSIM + "/"
print(path)

Load the simulated data.

In [ ]:
adata = scanpy.AnnData(pd.read_csv(path + nameSIM + "_" + n_genes + "_" + model + "_Ms.csv"))
# unspliced and spliced counts
adata.layers["Ms"] = pd.DataFrame(pd.read_csv(path + nameSIM + "_" + n_genes + "_" + model + "_Ms.csv"))
adata.layers["Mu"] = pd.DataFrame(pd.read_csv(path + nameSIM + "_" + n_genes + "_" + model + "_Mu.csv"))

# auxiliary files
adata.obs = pd.DataFrame(pd.read_csv(nameSIM + "_obs.csv"))
adata.var = pd.DataFrame(pd.read_csv(nameSIM + "_var.csv"))



**Basic preprocessing**


The pre-processing is skipped, since we have already simulated continuous data, following scVelo likelihood (or simulation structure). 
Avoiding this pre-processing here is in accordance with *scvelo.datasets.simulation* function.

In [ ]:
scv.pp.neighbors(adata)

In [ ]:
index = adata.var["index"]
adata.var_names = [str(i) for i in index]


**Dynamical Model**


In [ ]:
scv.tl.recover_dynamics(adata, fit_basal_transcription = True, var_names = "all") # We assume that there is basal transcription, since we have simulated alpha > 0

In [ ]:
scv.tl.velocity(adata, mode='dynamical', fit_offset = True)
scv.tl.velocity_graph(adata)
velocity = pd.DataFrame(np.asmatrix(adata.layers["velocity"]))

In [ ]:
scv.pl.velocity_embedding(adata, arrow_length=8, arrow_size=1, dpi=300, save = path + "/scVelo_velocityUmap.pdf" )

**Save data**

In [ ]:
pathToSave = path + "/res_sim_" + nameSIM + "_" + str(n_genes) + "genes_" + str(model) + "_scVelo_"
adata.write(pathToSave + ".h5ad")
adata.write_csvs(pathToSave + ".csv", skip_data=False)

In [ ]:
top_genes = adata.var['fit_likelihood'].sort_values(ascending=False).index[:300]
top_genes = pd.DataFrame(data=top_genes)
top_genes.to_csv(pathToSave + 'top_genes.csv', index=False)

# transcription, splicing and degradation rate and steady states
alpha = pd.DataFrame(np.asmatrix(adata.var["fit_alpha"])) 
alpha.to_csv(pathToSave + 'fit_alpha.csv', index=False)
beta = pd.DataFrame(np.asmatrix(adata.var["fit_beta"]))
beta.to_csv(pathToSave + 'fit_beta.csv', index=False)
gamma = pd.DataFrame(np.asmatrix(adata.var["fit_gamma"]))
gamma.to_csv(pathToSave + 'fit_gamma.csv', index=False)
fit_steady_u = pd.DataFrame(np.asmatrix(adata.var["fit_steady_u"]))
fit_steady_u.to_csv(pathToSave + 'fit_steady_u.csv', index=False)
fit_steady_s= pd.DataFrame(np.asmatrix(adata.var["fit_steady_s"]))
fit_steady_s.to_csv(pathToSave + 'fit_steady_s.csv', index=False)
fit_scaling = pd.DataFrame(np.asmatrix(adata.var["fit_scaling"]))
fit_scaling.to_csv(pathToSave + 'fit_scaling.csv', index=False)

# switching time
fit_u0 = pd.DataFrame(np.asmatrix(adata.var["fit_u0"]))
fit_u0.to_csv(pathToSave + 'fit_u0.csv', index=False)
fit_s0 = pd.DataFrame(np.asmatrix(adata.var["fit_s0"]))
fit_s0.to_csv(pathToSave + 'fit_s0.csv', index=False)
fit_t_ = pd.DataFrame(np.asmatrix(adata.var["fit_t_"]))
fit_t_.to_csv(pathToSave + 'fit_t_.csv', index=False)

# time
fit_t = pd.DataFrame(np.asmatrix(adata.layers["fit_t"]))
fit_t.to_csv(pathToSave + 'fit_t.csv', index=False)
fit_tau = pd.DataFrame(np.asmatrix(adata.layers["fit_tau"]))
fit_tau.to_csv(pathToSave + 'fit_tau.csv', index=False)
fit_tau_ = pd.DataFrame(np.asmatrix(adata.layers["fit_tau_"]))
fit_tau_.to_csv(pathToSave + 'fit_tau_.csv', index=False)
# velocity
velocity = pd.DataFrame(np.asmatrix(adata.layers["velocity"]))
velocity.to_csv(pathToSave + 'velocity.csv', index=False)
velocity_u = pd.DataFrame(np.asmatrix(adata.layers["velocity_u"]))
velocity_u.to_csv(pathToSave + 'velocity_u.csv', index=False)

Save some plots of the gene-specific dynamic.

In [ ]:
scv.pl.velocity(adata, [str(n) for n in range(20)] + ["20", "30", "40", "50", "60", "70", "80", "90", "100", "110", "120", "130", "140", "150", "160", "170", "180", "190", "200", "210", "220", "230", "240", "250", "260", "270", "280", "290", "300", "310", "320", "330", "340", "350", "360", "370", "380", "390", "400", "410", "420", "430", "440", "450", "460", "470", "480", "490", "500", "510", "520", "530", "540", "550", "560", "570", "580", "590", "600", "610", "620", "630", "640", "650", "660", "670", "680", "690", "700"], layers = 'Ms', dpi = 150, ncols = 2, save = pathToSave + '_dynamicGenes.pdf')